In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Overview
This lab demonstrates how to easily integrate generative AI features into your applications with just a few lines of code using pgvector, LangChain, and LLMs on Google Cloud.

We will build a sample Python application together that will be able to understand and respond to human language queries about the relational data stored in your PostgreSQL database. In fact, we will further push the creative limits of the application by teaching it to generate new content based on our existing dataset.

This lab utilizes an example of an e-commerce company that operates an online marketplace for buying and selling children's toys. The company aims to incorporate new generative AI experiences into its e-commerce applications for both buyers and sellers on the platform.

## The goals are:

(Usecase 1) For buyers: Build a new AI-powered hybrid search, where users can describe their needs in simple English text, along with regular filters (like price, etc.) (Usecase 2) For sellers: Add a new AI-powered content generation feature, where sellers will get auto-generated item description suggestions for new products that they want to add to the platform.

Dataset: The dataset for this lab has been sampled and created from a larger public retail dataset available at Kaggle. The sampled dataset used in this lab has only about 800 toy products, while the public dataset has over 370,000 products in different categories.

### Objective
At the end of this lab:

You will have a good understanding of how to use the pgvector extension to store and search vector embeddings in PostgreSQL. Learn more about vector embeddings.
You will get a hands-on experience with using the open-source LangChain framework to develop applications powered by large language models. LangChain makes it easier to develop and deploy applications against any LLM model in a vendor-agnostic manner.
You will learn about the powerful features in Google PaLM models made available through Vertex AI.

# [TODO] Add your H1 title heading here

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.10.13

## Overview

{TODO: Include a paragraph or two explaining what this example demonstrates, who should be interested in it, and what you need to know before you get started.}

Learn more about [web-doc-title](linkback-to-webdoc-page). {TODO: if more than one primary feature, add tag/linkback for each one}

### Objective

In this tutorial, you learn how to {TODO: Complete the sentence explaining briefly what you will learn from the notebook, such as
training, hyperparameter tuning, or serving}:

This tutorial uses the following Google Cloud ML services and resources:

- *{TODO: Add high level bullets for the services/resources demonstrated; e.g., Vertex AI Training}*


The steps performed include:

- *{TODO: Add high level bullets for the steps of performed in the notebook}*

### Dataset

{TODO: Include a paragraph with Dataset information and where to obtain it.} 

{TODO: Make sure the dataset is accessible to the public. **Googlers**: Add your dataset to the [public samples bucket](http://goto/cloudsamples#sample-storage-bucket) within gs://cloud-samples-data/vertex-ai, if it doesn't already exist there.}

### Costs 

{TODO: Update the list of billable products that your tutorial uses.}

This tutorial uses billable components of Google Cloud:

* Vertex AI
* {TODO: BigQuery}
* Cloud Storage

{TODO: Include links to pricing documentation for each product you listed above.
 NOTE: If you use BigQuery or Dataflow, you need to add this to the pricing.
}

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
{ TODO: [BigQuery pricing](https://cloud.google.com/bigquery/pricing), }
and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), 
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages required to execute this notebook. 

{TODO: Suggest using the latest major GA version of each package; i.e., --upgrade}

In [2]:
!pip install numpy pandas
!pip install pgvector
!pip install langchain langchain_google_vertexai transformers
!pip install google-cloud-aiplatform
!pip install psycopg2-binary
!pip install protobuf
!pip install shapely

INFO: pip is looking at multiple versions of google-api-core[grpc] to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 78.8 MB/s eta 0:00:00
  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-0.10.1
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 1.34.1
    Uninstalling google-api-core-1.34.1:
      Succes

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). {TODO: Update the APIs needed for your tutorial. Edit the API names, and update the link to append the API IDs, separating each one with a comma. For example, container.googleapis.com,cloudbuild.googleapis.com}

4. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

# Restart kernal


In [1]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

# Task 2. Download and load the dataset
An AlloyDB cluster named cymbal-alloy-cluster is configured in this lab. To begin, let's locate the AlloyDB cluster's IP address.

On the Google Cloud console title bar, type "AlloyDB" in the Search field, then click AlloyDB in the Products & Pages section.

Locate the cluster named cymbal-alloy-cluster, and the primary instance named cymbal-master-instance. The private IP address of this instance serves as your access point for utilizing AlloyDB throughout the lab.

Back in Vertex AI Workbench Notebook, import necessary libraries.

In [1]:
import os
import pandas as pd
import vertexai
from vertexai.language_models import TextEmbeddingModel
from vertexai.generative_models import GenerativeModel
from IPython.display import display, Markdown

from langchain_google_vertexai import VertexAIEmbeddings
import vertexai

PROJECT_ID = "qwiklabs-gcp-03-343e8b14b997"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

# Initialize Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=REGION)

In [ ]:
# PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
# ! gcloud config set project {PROJECT_ID}

In [9]:
# Run the following code snippet to import the psycopg2 library, which allows Python to interact with PostgreSQL databases, reads the CSV dataset into a pandas DataFrame, and finally saves the DataFrame to a table named products in the AlloyDB cluster.
import psycopg2

# Replace with your AlloyDB cluster credentials
cluster_ip_address = "10.107.0.2"
database_user = "postgres"
database_password = "postgres"

# Set environment variables for psql connection
os.environ["PGHOST"] = cluster_ip_address
os.environ["PGUSER"] = database_user
os.environ["PGPASSWORD"] = database_password

# Establish a connection to the database
try:
    conn = psycopg2.connect(
        host=cluster_ip_address,
        user=database_user,
        password=database_password
    )
    print("Connected to the database successfully!")
except Exception as e:
    print("Connection error:", e)
exit(1)

# Read the dataset from the URL
DATASET_URL = "https://github.com/GoogleCloudPlatform/python-docs-samples/raw/main/cloud-sql/postgres/pgvector/data/retail_toy_dataset.csv"
df = pd.read_csv(DATASET_URL)

# Select desired columns and drop missing values
df = df.loc[:, ["product_id", "product_name", "description", "list_price"]]
df = df.dropna()

# Save the DataFrame to the AlloyDB cluster
df.to_sql('products', con=f'postgresql://{cluster_ip_address}', if_exists='replace', index=False)

# Retrieve data from the 'products' table
cur = conn.cursor()
cur.execute("SELECT * FROM products")
results = cur.fetchall()

# Close the connection
conn.close()
tuplex=('product_id',
'product_name',
'description',
'list_price')
print(tuplex)
print(results[5])

Connected to the database successfully!
('product_id', 'product_name', 'description', 'list_price')
('74a695e3675efc2aad11ed73c46db29b', 'Slip N Slide Triple Racer with Slide Boogies', 'Triple Racer Slip and Slide with Boogie Boards. The unit is 16 foot long. The unit has 3 sliding lanes.', 37.21)


# Task 3. Generate Vector Embeddings using a Text Embedding Model
In this section, let's preprocess product descriptions, generate vector embeddings for them, and store the embeddings along with other relevant data in a PostgreSQL database table for downstream analysis or applications.

Run the following code snippet to import the RecursiveTextSplitter class from the LangChain library, which is used for splitting text into smaller chunks. Iterate through each row in the DataFrame df and extract the product ID and description from each row.

Then, we will split each description into smaller chunks and will create a dictionary for each chunk.

In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document

# Set up the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=[".", "\n"],
    chunk_size=500,
    chunk_overlap=0,
    length_function=len,
)

# Define the maximum number of documents to process
max_documents = 50  # Reduced limit to further control API usage
documents = []

# Create Document objects with product_id as metadata
for index, row in df.iterrows():
    product_id = row["product_id"]
    desc = row["description"]
    documents.append(Document(page_content=desc, metadata={"product_id": product_id}))

# Use the text splitter on a subset of documents (e.g., 40-50)
chunked = []
docs = text_splitter.split_documents(documents[40:max_documents])

# Collect split content along with product_id
for doc in docs:
    chunked.append({"product_id": doc.metadata["product_id"], "content": doc.page_content})

print(docs[1])
print('\nTotal  number of chuncks created:',len(chunked))


page_content='. Holds Up to 6 Decks Fun for the whole family! Smooth Rolling Action Heavy Plastic Holds Up to 6 DecksFun for the whole family!Smooth Rolling ActionHeavy Plastic' metadata={'product_id': '8a6d71be41e01b284294ec488508b414'}

Total  number of chuncks created: 34


### Run the following code snippet to process product descriptions from a dataset by splitting them into smaller chunks, sending them to Vertex AI for embedding generation, and storing the retrieved embeddings back into the data structure.

In [17]:
from langchain_google_vertexai import VertexAIEmbeddings
from google.cloud import aiplatform
import time

embeddings_service = VertexAIEmbeddings(model_name="textembedding-gecko")

# Helper function to retry failed API requests with exponential backoff.
def retry_with_backoff(func, *args, retry_delay=10, backoff_factor=2.5, **kwargs):  # Increased delay and backoff factor
    max_attempts = 10
    retries = 0
    for i in range(max_attempts):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"error: {e}")
            retries += 1
            wait = retry_delay * (backoff_factor**retries)
            print(f"Retry after waiting for {wait} seconds...")
            time.sleep(wait)

# Reduced batch size for API calls to manage quota limits
batch_size = 3
for i in range(0, len(chunked), batch_size):
    request = [x["content"] for x in chunked[i : i + batch_size]]
    response = retry_with_backoff(embeddings_service.embed_documents, request)
    # Store the retrieved vector embeddings for each chunk back.
    for x, e in zip(chunked[i : i + batch_size], response):
        x["embedding"] = e

# Store the generated embeddings in a pandas dataframe.
product_embeddings = pd.DataFrame(chunked)
product_embeddings.head()


Retrying vertexai.language_models._language_models._TextEmbeddingModel.get_embeddings in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: textembedding-gecko. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..
Retrying vertexai.language_models._language_models._TextEmbeddingModel.get_embeddings in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: textembedding-gecko. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..
Retrying vertexai.language_models._language_models._TextEmbeddingModel.get_embeddings in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: textembedding-gec

,product_id,content,embedding
0,8a6d71be41e01b284294ec488508b414,All of our productsWalmartply with internation...,"[-0.0019801377784460783, -0.037444762885570526..."
1,8a6d71be41e01b284294ec488508b414,. Holds Up to 6 Decks Fun for the whole family...,"[0.008327648043632507, -0.03802461177110672, -..."
2,9648838f5badebb9fc0b07f89cc29394,Better circulate water through your pool with ...,"[-0.045468609780073166, -0.0008376826299354434..."
3,9648838f5badebb9fc0b07f89cc29394,".25-inch fitting (11070), 2 strainer grids (11...","[-0.020975708961486816, 0.010413266718387604, ..."
4,9648838f5badebb9fc0b07f89cc29394,. Circulate water through your pool with the h...,"[-0.04218039661645889, -0.014087582007050514, ..."


### Run the following command to enable AlloyDB integration.

In [26]:
!PROJECT_ID=$(gcloud config get-value project) && \
PROJECT_NUMBER=$(gcloud projects list --filter="name=$PROJECT_ID" --format="value(PROJECT_NUMBER)") && \
gcloud projects add-iam-policy-binding $PROJECT_ID \
--member="serviceAccount:service-$PROJECT_NUMBER@gcp-sa-alloydb.iam.gserviceaccount.com" \
--role="roles/aiplatform.user"

ERROR: (gcloud.projects.add-iam-policy-binding) [329758193370-compute@developer.gserviceaccount.com] does not have permission to access projects instance [qwiklabs-gcp-03-343e8b14b997:setIamPolicy] (or it may not exist): Policy update access denied. This command is authenticated as 329758193370-compute@developer.gserviceaccount.com which is the active account specified by the [core/account] property.


In [25]:
!gcloud projects get-iam-policy $PROJECT_ID --format="json"


{
  "bindings": [
    {
      "members": [
        "serviceAccount:service-329758193370@gcp-sa-aiplatform.iam.gserviceaccount.com"
      ],
      "role": "roles/aiplatform.serviceAgent"
    },
    {
      "members": [
        "serviceAccount:service-329758193370@gcp-sa-aiplatform.iam.gserviceaccount.com"
      ],
      "role": "roles/aiplatform.user"
    },
    {
      "members": [
        "serviceAccount:service-329758193370@gcp-sa-alloydb.iam.gserviceaccount.com"
      ],
      "role": "roles/alloydb.serviceAgent"
    },
    {
      "members": [
        "serviceAccount:qwiklabs-gcp-03-343e8b14b997@qwiklabs-gcp-03-343e8b14b997.iam.gserviceaccount.com"
      ],
      "role": "roles/bigquery.admin"
    },
    {
      "members": [
        "serviceAccount:329758193370@cloudbuild.gserviceaccount.com"
      ],
      "role": "roles/cloudbuild.builds.builder"
    },
    {
      "members": [
        "serviceAccount:service-329758193370@gcp-sa-cloudbuild.iam.gserviceaccount.com"
      ],
      

#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [1]:
REGION = "us-central1"  # @param {type: "string"}

### Authenticate your Google Cloud account

The Cloud SDK, code and other libraries currently run as the service account identity of the Workbench Instance running this notebook.

**- Authenticate the Cloud SDK with your credentials :**

In [2]:
# ! gcloud auth login

**- Authenticate code and libraries with your credentials :**

In [ ]:
# ! gcloud auth application-default

**- Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

- *{Note to notebook author: For any user-provided strings that need to be unique (like bucket names or model ID's), append "-unique" to the end so proper testing can occur}*

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### Import libraries

In [ ]:
from google.cloud import aiplatform

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

{TODO: Include commands to delete individual resources below}

In [1]:
import os

# Delete endpoint resource
# e.g. `endpoint.delete()`

# Delete model resource
# e.g. `model.delete()`

# Delete Cloud Storage objects that were created
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI